In [24]:
!pip install chromaDB

  Using cached tokenizers-0.21.2-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached typer-0.16.0-py3-none-any.whl.metadata (15 kB)
  Using cached tomli-2.2.1-py3-none-any.whl.metadata (10 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached markdown_it_py-3.0.0-py3-n

## AI에서의 RAG System

- 인덱싱: 외부 데이터 소스를 전처리 한 후, 데이터를 나타내는 임베딩을 손쉽게 조회하도록 벡터 저장소에 저장
- 검색: 질문을 바탕으로 벡터 저장소에 보관된 관련 임베딩 및 데이터 추출
- 생성: 원래의 프롬프트와 RAG를 활용한 관련 문서를 종합하여 하나의 최종 프롬프트를 구성한 후, 이를 LLM에 전달 

## 1. Indexing 

In [40]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import chain
from dotenv import load_dotenv
import os

load_dotenv("../../.env")
api_key = os.getenv("OPENAI_API_KEY")

# 문서 로드
raw_document = TextLoader("./data/test.txt", encoding='utf-8').load()
raw_document


[Document(metadata={'source': './data/test.txt'}, page_content='인공지능(AI)은 21세기 가장 혁신적인 기술 중 하나로 손꼽히며, 우리의 삶과 사회 전반에 걸쳐 막대한 영향을 미치고 있습니다. AI의 역사는 1950년대 다트머스 회의에서 "인공지능"이라는 용어가 처음 사용되면서 시작되었습니다. 초기에는 주로 규칙 기반 시스템과 논리 추론에 중점을 두었으나, 계산 능력의 한계로 인해 "AI 겨울"이라는 침체기를 겪기도 했습니다. 그러나 2000년대 이후 딥러닝 기술의 발전과 빅데이터의 등장은 AI 연구에 새로운 활력을 불어넣었습니다. 특히 GPU와 같은 고성능 컴퓨팅 자원의 등장은 복잡한 신경망 모델을 훈련시키는 것을 가능하게 했습니다. 이러한 기술적 진보는 AI가 단순한 연구 단계를 넘어 실생활에 적용되는 시대를 열었습니다.\n\n현재 AI 기술은 머신러닝, 딥러닝, 자연어 처리(NLP), 컴퓨터 비전, 로보틱스 등 다양한 분야로 확장되었습니다. 머신러닝은 데이터를 기반으로 학습하여 패턴을 인식하고 예측하는 기술이며, 딥러닝은 여러 계층의 신경망을 통해 더욱 복잡한 특징을 학습하는 머신러닝의 한 분야입니다. 자연어 처리(NLP)는 컴퓨터가 인간의 언어를 이해하고 생성하며 번역하는 기술로, 챗봇, 음성 비서, 기계 번역 등에서 핵심적인 역할을 합니다. 컴퓨터 비전은 이미지를 분석하고 이해하여 객체 인식, 안면 인식, 자율 주행 등에 활용됩니다. 로보틱스는 AI를 로봇에 접목하여 자율적인 동작과 의사결정을 가능하게 합니다. 이러한 기술들은 서로 유기적으로 결합되어 더욱 강력한 AI 시스템을 구축하는 데 기여하고 있습니다. 특히, 강화 학습은 에이전트가 환경과 상호작용하며 보상을 통해 최적의 행동을 학습하는 방식으로, 게임 플레이나 로봇 제어 등에서 뛰어난 성과를 보여주고 있습니다.\n\nAI의 발전은 의료, 금융, 교육, 제조, 유통 등 거의 모든 산업 분야에 혁신을 가져오고 있습니다. 의료 분야에서는 질병 진단

In [42]:
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size = 500, chunk_overlap = 200
)

documents = text_splitter.split_documents(raw_document)
documents

[Document(metadata={'source': './data/test.txt'}, page_content='인공지능(AI)은 21세기 가장 혁신적인 기술 중 하나로 손꼽히며, 우리의 삶과 사회 전반에 걸쳐 막대한 영향을 미치고 있습니다. AI의 역사는 1950년대 다트머스 회의에서 "인공지능"이라는 용어가 처음 사용되면서 시작되었습니다. 초기에는 주로 규칙 기반 시스템과 논리 추론에 중점을 두었으나, 계산 능력의 한계로 인해 "AI 겨울"이라는 침체기를 겪기도 했습니다. 그러나 2000년대 이후 딥러닝 기술의 발전과 빅데이터의 등장은 AI 연구에 새로운 활력을 불어넣었습니다. 특히 GPU와 같은 고성능 컴퓨팅 자원의 등장은 복잡한 신경망 모델을 훈련시키는 것을 가능하게 했습니다. 이러한 기술적 진보는 AI가 단순한 연구 단계를 넘어 실생활에 적용되는 시대를 열었습니다.'),
 Document(metadata={'source': './data/test.txt'}, page_content='현재 AI 기술은 머신러닝, 딥러닝, 자연어 처리(NLP), 컴퓨터 비전, 로보틱스 등 다양한 분야로 확장되었습니다. 머신러닝은 데이터를 기반으로 학습하여 패턴을 인식하고 예측하는 기술이며, 딥러닝은 여러 계층의 신경망을 통해 더욱 복잡한 특징을 학습하는 머신러닝의 한 분야입니다. 자연어 처리(NLP)는 컴퓨터가 인간의 언어를 이해하고 생성하며 번역하는 기술로, 챗봇, 음성 비서, 기계 번역 등에서 핵심적인 역할을 합니다. 컴퓨터 비전은 이미지를 분석하고 이해하여 객체 인식, 안면 인식, 자율 주행 등에 활용됩니다. 로보틱스는 AI를 로봇에 접목하여 자율적인 동작과 의사결정을 가능하게 합니다. 이러한 기술들은 서로 유기적으로 결합되어 더욱 강력한 AI 시스템을 구축하는 데 기여하고 있습니다. 특히, 강화 학습은 에이전트가 환경과 상호작용하며 보상을 통해 최적의 행동을 학습하는 방식으로, 게임 플레이나 로봇 제어 등에서 뛰어난 성과를 보여주고 있습니다.'),
 Docume

### ChromaDB 활용

In [43]:
embedding_model = OpenAIEmbeddings(api_key=api_key)

persist_directory = "./chroma_db_data" # ChromaDB가 데이터를 저장할 로컬 파일 시스템의 경로를 지정 

db = Chroma.from_documents(
  documents, 
  embedding_model,
  persist_directory=persist_directory
)

### SQLite3로 ChromaDB 탐색

In [44]:
import sqlite3
import json

# chroma.sqlite3 경로 
data_path = "./chroma_db_data/chroma.sqlite3"

try:
  conn = sqlite3.connect(data_path)
  cursor = conn.cursor() # db 실행
  
  print(f"테이블 목록")
  # sqlite_master 테이블에서 type이 table인 것의 name 선택 
  cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
  tables = cursor.fetchall()
  # table 컬럼 확인 
  for table in tables:
        print(table[0])
  print("\n--- 'embeddings' 테이블 내용 (처음 3개 행) ---")
    # 보통 .tables 명령어로 정확한 테이블명을 확인 후 사용
    
  cursor.execute("SELECT * FROM embeddings LIMIT 3;")
  rows = cursor.fetchall()

  for row in rows:
      #embedding_str = row
      print(row)
    
      #print(f"Embedding (first 20 chars): {embedding_str[:20]}...")
      print("-" * 30)

except sqlite3.Error as e:
  print(f"데이터베이스 연결 또는 쿼리 오류: {e}")
finally:
  if 'conn' in locals() and conn:
      conn.close()

테이블 목록
migrations
acquire_write
collection_metadata
segment_metadata
tenants
databases
collections
maintenance_log
segments
embeddings
embedding_metadata
max_seq_id
embedding_fulltext_search
embedding_fulltext_search_data
embedding_fulltext_search_idx
embedding_fulltext_search_content
embedding_fulltext_search_docsize
embedding_fulltext_search_config
embeddings_queue
embeddings_queue_config

--- 'embeddings' 테이블 내용 (처음 3개 행) ---
(1, '00094581-86e2-4dbe-acf9-c7520ff30d45', '15ece929-9c41-4391-946d-2a8efd257b57', 1, '2025-07-28 00:41:01')
------------------------------
(2, '00094581-86e2-4dbe-acf9-c7520ff30d45', '76a06d57-824f-4060-b68a-8634f905bf08', 2, '2025-07-28 00:41:01')
------------------------------
(3, '00094581-86e2-4dbe-acf9-c7520ff30d45', '636992f3-0767-46c0-94d1-0c31fc91962e', 3, '2025-07-28 01:19:37')
------------------------------


## 2. Retriever
- 입력된 질의를 임베딩 형태로 변환
- 벡터 저장소에서 사용자 질의와 가장 유사한 임베딩 산출

In [45]:
# 질의 임베딩 및 벡터 저장소를 통해 실행하는 유사도 검색 계산 로직 추상화 
retriever = db.as_retriever() 

query = "컴퓨터 비전 기술은 무엇인가요?"

# 관련 문서를 받아옴 
print(retriever.invoke(query))

[Document(metadata={'source': './data/test.txt'}, page_content='현재 AI 기술은 머신러닝, 딥러닝, 자연어 처리(NLP), 컴퓨터 비전, 로보틱스 등 다양한 분야로 확장되었습니다. 머신러닝은 데이터를 기반으로 학습하여 패턴을 인식하고 예측하는 기술이며, 딥러닝은 여러 계층의 신경망을 통해 더욱 복잡한 특징을 학습하는 머신러닝의 한 분야입니다. 자연어 처리(NLP)는 컴퓨터가 인간의 언어를 이해하고 생성하며 번역하는 기술로, 챗봇, 음성 비서, 기계 번역 등에서 핵심적인 역할을 합니다. 컴퓨터 비전은 이미지를 분석하고 이해하여 객체 인식, 안면 인식, 자율 주행 등에 활용됩니다. 로보틱스는 AI를 로봇에 접목하여 자율적인 동작과 의사결정을 가능하게 합니다. 이러한 기술들은 서로 유기적으로 결합되어 더욱 강력한 AI 시스템을 구축하는 데 기여하고 있습니다. 특히, 강화 학습은 에이전트가 환경과 상호작용하며 보상을 통해 최적의 행동을 학습하는 방식으로, 게임 플레이나 로봇 제어 등에서 뛰어난 성과를 보여주고 있습니다.'), Document(metadata={'source': './data/test.txt'}, page_content='인공지능(AI)은 21세기 가장 혁신적인 기술 중 하나로 손꼽히며, 우리의 삶과 사회 전반에 걸쳐 막대한 영향을 미치고 있습니다. AI의 역사는 1950년대 다트머스 회의에서 "인공지능"이라는 용어가 처음 사용되면서 시작되었습니다. 초기에는 주로 규칙 기반 시스템과 논리 추론에 중점을 두었으나, 계산 능력의 한계로 인해 "AI 겨울"이라는 침체기를 겪기도 했습니다. 그러나 2000년대 이후 딥러닝 기술의 발전과 빅데이터의 등장은 AI 연구에 새로운 활력을 불어넣었습니다. 특히 GPU와 같은 고성능 컴퓨팅 자원의 등장은 복잡한 신경망 모델을 훈련시키는 것을 가능하게 했습니다.\n\n현재 AI 기술은 머신러닝, 딥러닝, 자연어 처리(NLP), 컴퓨터 비전, 로보틱스 등 다양한 분야로 확장

In [51]:
retriever = db.as_retriever(search_kwargs = {"k" : 2})
query = "군사 차원에서 인공지능에 대해 알려줘?"

docs = retriever.invoke(query)
docs

[Document(metadata={'source': './data/test.txt'}, page_content='인공지능(AI)은 21세기 가장 혁신적인 기술 중 하나로 손꼽히며, 우리의 삶과 사회 전반에 걸쳐 막대한 영향을 미치고 있습니다. AI의 역사는 1950년대 다트머스 회의에서 "인공지능"이라는 용어가 처음 사용되면서 시작되었습니다. 초기에는 주로 규칙 기반 시스템과 논리 추론에 중점을 두었으나, 계산 능력의 한계로 인해 "AI 겨울"이라는 침체기를 겪기도 했습니다. 그러나 2000년대 이후 딥러닝 기술의 발전과 빅데이터의 등장은 AI 연구에 새로운 활력을 불어넣었습니다. 특히 GPU와 같은 고성능 컴퓨팅 자원의 등장은 복잡한 신경망 모델을 훈련시키는 것을 가능하게 했습니다. 이러한 기술적 진보는 AI가 단순한 연구 단계를 넘어 실생활에 적용되는 시대를 열었습니다.'),
 Document(metadata={'source': './data/test.txt'}, page_content='AI의 군사적 활용 역시 국제사회의 주요 관심사입니다. 자율 살상 무기(LAWS)의 개발과 배치는 인류의 통제력을 벗어난 심각한 위협이 될 수 있다는 우려가 제기되고 있습니다. 이에 대한 국제적인 합의와 규제 마련이 시급하며, AI의 평화적 사용을 위한 노력이 병행되어야 합니다. AI가 인간의 인지 능력을 뛰어넘는 특이점(Singularity)에 도달할 것이라는 논의도 있지만, 이는 아직 먼 미래의 가설로 여겨집니다. 중요한 것은 현재 우리가 마주한 AI 기술의 실제적인 영향과 잠재적 위험을 이해하고, 이를 통제하고 관리할 수 있는 능력을 키우는 것입니다. AI는 단순히 도구가 아니라, 사회 구조와 인간의 삶의 방식 자체를 변화시키는 강력한 힘을 가지고 있기 때문입니다.')]

## 3. Generator

In [52]:
# retriever = db.as_retriever(search_kwargs = {"k" : 3})
# from_template는 심플한 문자열이 채워진 단일 객체 리스트 
prompt = ChatPromptTemplate.from_template(
''' 다음 컨텍스트만 사용하여 질문에 응답하시오.
컨텍스트: {context}

질문: {question}
'''
)

llm = ChatOpenAI(
  model = "gpt-4o-mini",
  api_key=api_key,
  temperature = 0.7,
)

llm_chain = prompt | llm 
llm_chain

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template=' 다음 컨텍스트만 사용하여 질문에 응답하시오.\n컨텍스트: {context}\n\n질문: {question}\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000027BB442D090>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000027BB442EAA0>, root_client=<openai.OpenAI object at 0x0000027BB442EC50>, root_async_client=<openai.AsyncOpenAI object at 0x0000027BB442F7F0>, model_name='gpt-4o-mini', temperature=0.7, model_kwargs={}, openai_api_key=SecretStr('**********'))

In [54]:
result = llm_chain.invoke({'context':docs, "question": query})
result.content

'군사 차원에서 인공지능(AI)의 활용은 국제 사회에서 주요 관심사로 떠오르고 있습니다. 특히 자율 살상 무기(LAWS)의 개발과 배치는 인류의 통제력을 벗어난 심각한 위협이 될 수 있다는 우려가 제기되고 있습니다. 이러한 문제를 해결하기 위해 국제적인 합의와 규제 마련이 시급하며, 동시에 AI의 평화적 사용을 위한 노력도 필요합니다. AI가 인간의 인지 능력을 뛰어넘는 특이점(Singularity)에 도달할 가능성에 대한 논의도 있지만, 이는 아직 먼 미래의 가설로 여겨지고 있습니다. 따라서 현재 우리가 마주한 AI 기술의 실제적인 영향과 잠재적 위험을 이해하고, 이를 통제하고 관리할 수 있는 능력을 키우는 것이 중요합니다. AI는 단순한 도구가 아니라 사회 구조와 인간의 삶의 방식을 변화시키는 강력한 힘을 가지고 있기 때문입니다.'

### Retriever 로직 캡슐화
- `@chain`: 파이썬 데코레이터를 통해 함수를 러너블 체인으로 전환하여 `invoke()`나 `batch()` 메서드를 사용할 수 있게 만드는 것. 

In [58]:
retriever = db.as_retriever(search_kwargs = {"k" : 2})

prompt = ChatPromptTemplate.from_template(
''' 다음 컨텍스트만 사용하여 질문에 응답하시오.
컨텍스트: {context}

질문: {question}
'''
)

llm = ChatOpenAI(
  model = "gpt-4o-mini",
  api_key=api_key,
  temperature = 0.7,
)

@chain
def qa(input):
  # input에 대해 관련 문서 검색 
  docs = retriever.invoke(input)
  # 프롬프트 포멧 
  formatted = prompt.invoke({'context': docs, "question" : input})
  answer = llm.invoke(formatted)
  return answer

result = qa.invoke(query)
result.content

'군사 차원에서 인공지능(AI)의 활용은 국제사회의 주요 관심사 중 하나입니다. 특히 자율 살상 무기(LAWS)의 개발과 배치는 인류의 통제력을 벗어난 심각한 위협이 될 수 있다는 우려가 제기되고 있습니다. 이러한 상황에서는 국제적인 합의와 규제 마련이 시급하며, AI의 평화적 사용을 위한 노력도 병행되어야 합니다. \n\n또한, AI가 인간의 인지 능력을 뛰어넘는 특이점(Singularity)에 도달할 것이라는 논의도 있지만, 이는 아직 먼 미래의 가설로 여겨집니다. 중요한 것은 현재 우리가 마주한 AI 기술의 실제적인 영향과 잠재적 위험을 이해하고, 이를 통제하고 관리할 수 있는 능력을 키우는 것입니다. AI는 단순한 도구가 아니라 사회 구조와 인간의 삶의 방식 자체를 변화시키는 강력한 힘을 가지고 있어, 군사 분야에서도 그 영향력이 크다고 할 수 있습니다.'

## 4. Query Transformation
기본 RAG 시스템은 사용자가 입력한 쿼리의 품질에 과도하게 영향을 받는데, 사용자마다 입력 품질이 다를 때 그 입력을 추상적이나 구체적으로 변환하는 전략이다.

### 4.1 Rewrite-Retrieve-Read(RRR) 전략 
검색을 수행하기 전에 LLM에 사용자의 쿼리를 재작성하도록 프롬프트 전송하는 방법

In [69]:
@chain
def qa(input):
  docs = retriever.invoke(input)
  formatted_template = prompt.invoke({"context" : docs, "question" : input})
  answer = llm.invoke(formatted_template)
  
  return answer

# 불필요한 정보를 담은 query 전달 
query = "아침 6시에 일어나서 준비를 하고 외출을 했습니다. 아침은 먹지 않았어요. AI는 어떤 것인가요?"

result = qa.invoke(query)
result.content

'AI(인공지능)는 21세기 가장 혁신적인 기술 중 하나로, 우리의 삶과 사회 전반에 걸쳐 막대한 영향을 미치고 있습니다. AI의 역사는 1950년대 다트머스 회의에서 "인공지능"이라는 용어가 처음 사용되면서 시작되었으며, 초기에는 주로 규칙 기반 시스템과 논리 추론에 중점을 두었습니다. 그러나 2000년대 이후 딥러닝 기술의 발전과 빅데이터의 등장이 AI 연구에 새로운 활력을 불어넣어, AI는 실생활에 적용되는 시대를 맞이하게 되었습니다. AI는 단순한 도구가 아니라, 사회 구조와 인간의 삶의 방식 자체를 변화시키는 강력한 힘을 가지고 있습니다.'

In [ ]:
### RRR 구현 

rewrite_prompt = ChatPromptTemplate.from_template(
'''
웹 검색 엔진이 주어진 질문에 답할 수 있도록 더 나은 한글 검색어를 제공하세요. 쿼리는 \'**'\로 끝내세요

질문: {x}

답변: 
'''
)

def parse_rewriter_output(message):
  return message.content.strip('\'').strip('**')

# 재작성 chain 정의 
rewriter = rewrite_prompt | llm | parse_rewriter_output

# RRR 로직 구현 
@chain
def qa_rrr(input):
  new_query = rewriter.invoke(input)
  print(f"재작성한 쿼리: {new_query}")
  docs = retriever.invoke(new_query)
  formatted = prompt.invoke({"context" : docs, "question" : input})
  
  answer = llm.invoke(formatted)
  return answer 

result = qa_rrr.invoke(query)
result.content

재작성한 쿼리: AI의 정의와 역할은 무엇인가요\


'AI(인공지능)는 인간의 지능을 모방하여 다양한 작업을 수행할 수 있는 기술입니다. AI는 제너레이티브 AI와 멀티모달 AI와 같은 여러 형태로 발전하고 있으며, 텍스트, 이미지, 오디오 등 다양한 콘텐츠를 생성하고 이해하는 능력을 가지고 있습니다. AI는 또한 설명 가능한 AI(XAI)를 통해 의사결정 과정을 투명하게 이해할 수 있도록 도와주며, 윤리와 안전성에 대한 논의도 중요합니다. AI는 사회 구조와 인간의 삶의 방식을 변화시킬 수 있는 강력한 힘을 지니고 있습니다.'

### 4.2 Multi Query Search
초기 쿼리를 바탕으로 LLM에 여러 개의 쿼리를 생성하도록 지시한 후, 데이터 소스에서 각 쿼리에 대한 병렬 검색 수행 

In [73]:
perspectives_prompt = ChatPromptTemplate.from_template(
  '''
  당신은 AI 언어 모델 어시스턴트입니다. 주어진 사용자 질문의 다섯 가지 버전을 생성하여 벡터 데이터베이스 내에서 관련 문서를 검색하세요
  사용자 질문에 대한 다양한 관점을 생성함으로써 사용자가 거리 기반 유사도 검색의 한계를 극복할 수 있도록 돕는 것이 목표입니다. 
  이러한 대체 질문을 개행으로 구분하여 제공하세요
  원래 질문: {question}
  '''
)

def parse_queries_output(message):
  return message.content.split("\n")


query_gen = perspectives_prompt | llm | parse_queries_output
query_gen

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='\n  당신은 AI 언어 모델 어시스턴트입니다. 주어진 사용자 질문의 다섯 가지 버전을 생성하여 벡터 데이터베이스 내에서 관련 문서를 검색하세요\n  사용자 질문에 대한 다양한 관점을 생성함으로써 사용자가 거리 기반 유사도 검색의 한계를 극복할 수 있도록 돕는 것이 목표입니다. \n  이러한 대체 질문을 개행으로 구분하여 제공하세요\n  원래 질문: {question}\n  '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000027BB4DAE1D0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000027BBC785840>, root_client=<openai.OpenAI object at 0x0000027BB4DADF60>, root_async_client=<openai.AsyncOpenAI object at 0x0000027BB404C700>, model_name='gpt-4o-mini', temperature=0.7, model_kwargs={}, openai_api_key=SecretStr('**********'))
| RunnableLambda(parse_queries_output)

In [74]:
# query_gen에서 생성된 쿼리 목록을 활용하여 관련성이 높은 문서를 뽑아낸 후 중복 제거 
def get_unique_union(document_lists):
  deduped_docs = {
    # document_lists 내부의 sublist 내부의 doc를 추출하여 각 문서의 내용 추출
    # 딕셔너리의 키는 유일성을 만족해야하므로 중복 제거 로직 
    doc.page_content: doc for sublist in document_lists for doc in sublist
  }
  
  return list(deduped_docs.values())

# batch를 활용하여 생성된 모든 쿼리를 병렬로 실행 
retrieval_chain = query_gen | retriever.batch | get_unique_union

In [77]:
prompt = ChatPromptTemplate.from_template(
''' 다음 컨텍스트만 사용하여 질문에 응답하시오.
컨텍스트: {context}

질문: {question}
'''
)

query = "군사 차원에서 인공지능에 대해 알려줘"

@chain
def multi_query_qa(input):
  docs = retrieval_chain.invoke(input)
  formatted = prompt.invoke(
    {"context" : docs, "question": input}
  )
  answer = llm.invoke(formatted)
  return answer
print("다중 커리 검색\n")
result = multi_query_qa.invoke(query)

result.content

다중 커리 검색



'AI의 군사적 활용은 국제사회의 주요 관심사로 떠오르고 있습니다. 자율 살상 무기(LAWS)의 개발과 배치는 인류의 통제력을 벗어난 심각한 위협이 될 수 있다는 우려가 제기되고 있으며, 이에 대한 국제적인 합의와 규제 마련이 시급합니다. 이러한 논의와 함께 AI의 평화적 사용을 위한 노력도 병행되어야 합니다. AI가 인간의 인지 능력을 뛰어넘는 특이점(Singularity)에 도달할 것이라는 논의도 있지만, 이는 아직 먼 미래의 가설로 간주됩니다. 현재 중요한 것은 AI 기술의 실제적인 영향과 잠재적 위험을 이해하고, 이를 효과적으로 통제하고 관리할 수 있는 능력을 키우는 것입니다. AI는 단순한 도구가 아니라 사회 구조와 인간의 삶의 방식을 변화시키는 강력한 힘을 지니고 있습니다.'

### Query Routing 
벡터 저장소만 사용하는 편이 유용하더라도, 필요한 데이터는 RDB를 비롯한 다른 DB에 존재할 수 있는데, 이때 사용자의 질문을 바탕으로 쿼리를 라우팅하여 적절한 데이터 소스로 전달해 관련 문서를 검색하는 방법

- **논리적 라우팅(logical routing)**: LLM에게 활용할 수 있는 여러 데이터 소스에 관한 정보를 제공해주어 쿼리를 토대로 적합한 데이터 소스를 선택하도록 하는 방식으로, 정의된 데이터 소스 목록이 존재하고 해당 소스로부터 관련 데이터를 검색하여 LLM에 적용해 정확한 결과물을 생성할 수 있을 때 논리적 라우팅 방식이 적합하다. 

- **의미론적 라우팅(semantic routing)**: 다양한 데이터 소스를 대표하는 프롬프트를 준비해 임베딩하고, 쿼리에 벡터 유사도 검색을 수행하여 가장 유사한 프롬프트를 검색하는 방법

### 논리적 라우팅 

In [ ]:
from typing import Literal # 둘 중 하나의 값만 가질 수 있는 타입 

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda
from pydantic import BaseModel, Field 

# 데이터 모델 
# json 형식으로 출력 
class RouteQuery(BaseModel):
  # Field는 데이터 모델에 추가적인 정보 제공 
  datasource: Literal['python_docs','js_docs'] = Field(
    ..., # 이 필드가 필수(required)임을 나타냄 
    description = "Given a user question, choose which dataource would be most relevant for answering their question",
  )
  
llm = ChatOpenAI(
  model = "gpt-4o-mini",
  api_key = api_key,
  temperature=0.7
)

# 주어진 스키마와 일치하도록 형식화된 출력을 반환하는 모델 래퍼 
structured_llm = llm.with_structured_output(RouteQuery)

system = """
당신은 사용자 질문을 적절한 데이터 소스로 라우팅하는 전문가입니다. 질문이 지목하는 프로그래밍 언어에 따라 해당 데이터 소스로 라우팅 하세요
"""

prompt = ChatPromptTemplate.from_messages(
  [('system', system), ('human', '{question}')]
)

# 라우터 정의 
router = prompt | structured_llm

In [79]:
question = '''이 코드가 안 돌아가는 이유를 설명해줘
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
  'human', 'speak in {language}'
])
prompt.invoke('french')
'''

result = router.invoke({"question" : question})
print(f"\nRouting to {result}")


Routing to datasource='python_docs'


In [81]:
## 추출된 값을 전달해서 라우팅 선택 함수 정의 

def choose_route(result):
  if 'python_docs' in result.datasource.lower():
    return f'chain for python_docs'
  else:
    return 'chain for js_docs'

# RunnableLambda을 통해 python 함수를 LCEL 체인에 통합하여 구성 
full_chain = router | RunnableLambda(choose_route)

result = full_chain.invoke({'question' : question})
print(f"\n choose route: {result}")


 choose route: chain for python_docs


### 의미론적 라우팅

In [89]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import chain
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

# 다양한 데이터 소스를 대표하는 프롬프트를 미리 생성 

physics_template = """ 당신은 매우 똑똑한 물리학 교수입니다.
당신은 물리학에 대한 질문에 간결하고 쉽게 이해할 수 있는 방식으로 대답하는 데 뛰어납니다.
당신이 질문에 대한 답을 모를 땐 모른다고 답변합니다.
다음 질문에 대답하시오: {query}
"""

sports_template = """당신은 매우 똑똑한 스포츠 지도자입니다.
당신은 스포츠와 관련된 지식이나 규칙 등을 쉽게 이해할 수 있는 방식으로 대답하는 데 뛰어납니다. 
당신이 질문에 대한 답으 모를 땐 모른다고 답변합니다.
다음 질문에 대답하시오: {query}
"""

embeddings = OpenAIEmbeddings()
prompt_templates = [physics_template, sports_template]

# 프롬프트 템플릿을 임베딩 
prompt_embeddings = embeddings.embed_documents(prompt_templates)


@chain

def prompt_router(query):
  # 입력 받은 query 임베딩 
  # embedding 차원은 동일해야되며, query와 같이 단일 문자열을 받는 경우 embed_query메서드를 활용 
  query_embeddings = embeddings.embed_query(query)
  similarity = cosine_similarity([query_embeddings], prompt_embeddings)[0]
  print(f"유사도: {similarity}")
  most_similar_template = prompt_templates[similarity.argmax()] 
  
  print(f"스포츠 프롬프트 사용" if most_similar_template == sports_template else "물리 프롬프트 사용")
  
  return PromptTemplate.from_template(most_similar_template)

semantic_router = (prompt_router | ChatOpenAI() | StrOutputParser())

result = semantic_router.invoke("축구는 무엇인가요?")

print(f"\n 의미론적 라우팅 결과 : {result}")
  

유사도: [0.75641031 0.82287172]
스포츠 프롬프트 사용

 의미론적 라우팅 결과 : 축구는 공을 발로 차서 상대팀 골대에 넣는 경기로, 11명으로 이루어진 두 팀이 경기를 진행하는 스포츠입니다.


## Query construction

자연어 쿼리를 사용 중인 데이터베이스나 데이터 소스의 쿼리 언어로 변환하는 과정 

- 텍스트 to 메타데이터 필터: 대부분의 벡터 저장소는 메타데이터를 토대로 벡터 검색 범위를 제한하는 기능을 갖추고 있으므로 쿼리 호출에서 필터 표현식 지정이 가능하다.
- 텍스트 to SQL


### Text-to-metadata

In [92]:
!pip install lark

In [2]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains.query_constructor.schema import AttributeInfo # 데이터 소스 속성 정보 
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_community.vectorstores import Chroma 


fields = [
  AttributeInfo(
    name = 'genre',
    description = '영화 장르',
    type = 'string or list[string]',
  ),
  AttributeInfo(
    name = 'year',
    description = '영화 개봉 연도',
    type = 'integer',
  ),AttributeInfo(
    name = 'director',
    description = '영화 감독',
    type = 'string',
  ),AttributeInfo(
    name = 'rating',
    description = '영화 평점 1-10점',
    type = 'float',
  ),
]

persist_directory = "./chroma_db_data"

vectorstore = Chroma(
  persist_directory = persist_directory,
  embedding_function = OpenAIEmbeddings()
)
# 디버깅 
print(f"ChromaDB가 '{persist_directory}' 경로에서 성공적으로 로드되었습니다.")
print(f"로드된 문서의 개수: {vectorstore._collection.count()}개")

description = "영화에 대한 간략한 정보"

llm = ChatOpenAI(model = "gpt-4o-mini", temperature = 0)

# 벡터 스토오와 LLM을 사용하여 벡터 스토어 쿼리를 생성하는 검색기 
retriever = SelfQueryRetriever.from_llm(llm, 
                                        vectorstore, 
                                        document_contents = description, 
                                        metadata_field_info = fields)

# 필터 적용 
print(retriever.invoke("미래 AI 관련된 내용만 보고싶어"))
print("\n")

ChromaDB가 './chroma_db_data' 경로에서 성공적으로 로드되었습니다.
로드된 문서의 개수: 10개
[Document(metadata={'source': './data/test.txt'}, page_content='미래의 AI는 더욱 인간과 유사한 상호작용이 가능한 방향으로 발전할 것으로 예상됩니다. 제너레이티브 AI(생성형 AI)는 텍스트, 이미지, 오디오 등 다양한 형태의 콘텐츠를 스스로 생성하는 능력을 보여주며, 이는 창의적인 산업과 개인의 생산성을 혁신할 잠재력을 가지고 있습니다. 이와 함께 멀티모달 AI는 텍스트, 이미지, 음성 등 여러 형태의 데이터를 동시에 이해하고 처리하여 더욱 풍부한 상호작용을 가능하게 할 것입니다. 또한, 설명 가능한 AI(XAI)에 대한 연구가 활발히 진행되어 AI의 의사결정 과정을 투명하게 이해하고 신뢰성을 높이는 데 기여할 것입니다. AI 윤리와 안전성 또한 중요한 화두로 떠오르고 있으며, AI가 사회에 미치는 긍정적 영향을 극대화하고 부정적 영향을 최소화하기 위한 노력이 계속될 것입니다. 궁극적으로 AI는 인간의 지능을 보완하고 확장하여 인류의 삶의 질을 향상시키는 데 기여할 것입니다. 우리는 AI와 공존하며 새로운 시대를 열어갈 준비를 해야 합니다.'), Document(metadata={'source': './data/test.txt'}, page_content='인공지능(AI)은 21세기 가장 혁신적인 기술 중 하나로 손꼽히며, 우리의 삶과 사회 전반에 걸쳐 막대한 영향을 미치고 있습니다. AI의 역사는 1950년대 다트머스 회의에서 "인공지능"이라는 용어가 처음 사용되면서 시작되었습니다. 초기에는 주로 규칙 기반 시스템과 논리 추론에 중점을 두었으나, 계산 능력의 한계로 인해 "AI 겨울"이라는 침체기를 겪기도 했습니다. 그러나 2000년대 이후 딥러닝 기술의 발전과 빅데이터의 등장은 AI 연구에 새로운 활력을 불어넣었습니다. 특히 GPU와 같은 고성능 컴퓨팅 자원의 등장은 복잡한 신경망 모델을 

### Text-to-SQL

LLM으로 사용자의 query를 SQL 쿼리로 변환할 수는 있지만, 오류가 발생할 여지가 존재하므로 다음과 같은 방법을 사용한다.

- **데이터베이스 설명**: LLM에 정확한 데이터베이스 설명을 제공하여 SQL쿼리를 명확하게 만든다. 각 테이블에 대해 컬럼명과 자료형을 포함한 CREATE TABLE 을 LLM에 제공하고 테이블의 일부 예시 행을 제공한다. 

- **퓨샷 예시**: 프롬프트에 질문과 쿼리 간의 대응 예시 몇 가지를 첨부하여 쿼리 생성의 정확성을 높일 수 있다.

In [4]:
import requests
import sqlite3
import os

# SQL 파일 다운로드 URL
sql_url = "https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql"

output_path = "data/"
db_filename = os.path.join(output_path, "Chinook.db")

try:
    if not os.path.exists(output_path):
      os.makedirs(output_path)
      print(f"'{output_path}' 디렉터리를 생성했습니다.")
      
    # SQL 내용 다운로드
    print(f"'{sql_url}'에서 SQL 파일 다운로드 중...")
    response = requests.get(sql_url)
    response.raise_for_status() # HTTP 오류가 발생하면 예외를 발생시킵니다.
    sql_content = response.text # response 객체 자체는 HTML body이므로 텍스트만 추출하여 저장 
    print("다운로드 완료.")

    # SQLite 데이터베이스 연결 및 SQL 실행
    print(f"'{db_filename}' 데이터베이스 생성 및 SQL 실행 중...")
    conn = sqlite3.connect(db_filename) # DB 연결, 해당 filename의 DB가 존재하지 않는 경우 새로 생성 
    cursor = conn.cursor()

    # SQL 내용을 실행합니다.
    # 여러 개의 SQL 문이 있을 수 있으므로 executescript를 사용합니다.
    cursor.executescript(sql_content)
    conn.commit()
    conn.close()
    print(f"'{db_filename}' 데이터베이스가 성공적으로 생성되었습니다.")

except requests.exceptions.RequestException as e:
    print(f"SQL 파일 다운로드 중 오류 발생: {e}")
except sqlite3.Error as e:
    print(f"SQLite 데이터베이스 작업 중 오류 발생: {e}")
except Exception as e:
    print(f"예상치 못한 오류 발생: {e}")

'https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql'에서 SQL 파일 다운로드 중...
다운로드 완료.
'data/Chinook.db' 데이터베이스 생성 및 SQL 실행 중...
'data/Chinook.db' 데이터베이스가 성공적으로 생성되었습니다.


In [ ]:
from langchain_community.tools import QuerySQLDataBaseTool
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

# LLM에 Database의 스키마를 잘 이해할 수 있게 도움, 에이전트 또는 체인과 통합 가능 
db = SQLDatabase.from_uri("sqlite:///data/Chinook.db")
print(db.get_usable_table_names())

['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


In [ ]:
llm = ChatOpenAI(model = 'gpt-4o-mini', temperature = 0.7)

# 질문을 SQL 쿼리로 변환 
write_query = create_sql_query_chain(llm, db)

# SQL 쿼리 실행 
execute_query = QuerySQLDataBaseTool(db = db)

combined_chain = write_query | execute_query

result = combined_chain.invoke({"question":"직원(employee)은 모두 몇 명인가요?"})
result

'Error: (sqlite3.OperationalError) near "SQLQuery": syntax error\n[SQL: SQLQuery: SELECT COUNT(*) AS "EmployeeCount" FROM "Employee";]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)'

In [ ]:
from langchain_core.runnables import RunnableLambda 

# SQL 쿼리 정제 함수 정의 
def clean_sql_output(sql_string):
    if sql_string.strip().startswith("SQL"):
        # 접두사를 제외한 인덱스부터 문자열 추출 
        # SQLQuery를 제외한 후 ":"도 제거해야하므로 + 1
        sql_string = sql_string.strip()[len('SQLQuery') + 1:].strip()
        
    sql_string = sql_string.strip()
    return sql_string

full_chain = write_query | RunnableLambda(clean_sql_output) | execute_query

result = full_chain.invoke({"question": "직원(employee)은 모두 몇 명인가요?"})
result

'[(8,)]'